# Final Project Phase 2 Summary

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source: [Global Power Plant Database](https://datasets.wri.org/dataset/globalpowerplantdatabase) and [Global Power Plant Emissions Database](http://meicmodel.org/?page_id=91&lang=en)
*   Web Collection #1 Source: [WhatToMine Coins](https://whattomine.com/coins.json) and [WhatToMine GPUs](https://whattomine.com/gpus)
*   Web Collection #2 Source: [CoinGecko](https://www.coingecko.com/en/apiBlo)



## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from pprint import pprint
import requests, re

In [3]:
def global_power():
  # IMPORT FILES
  generation_path = "data\global-power-plants\global_power_plant_database.csv"
  emission_path = "data\global-power-plants\global_power_emissions_database.xlsx"
  
  # power generation csv
  with open(generation_path, encoding='utf8') as fin:
    ppg = pd.read_csv(fin, low_memory=False)
  
  # power plant emissions xlsx
  ppe = pd.read_excel(emission_path, sheet_name='GPED_v1.0_Plant Level', skiprows=0, header=1)

  # removing unwanted data
  unwanted_columns = ['latitude',
                      'longitude',
                      'other_fuel1',
                      'other_fuel2',
                      'other_fuel3',
                      'commissioning_year',
                      'gppd_idnr',
                      'owner',
                      'source',
                      'url',
                      'geolocation_source',
                      'wepp_id',
                      'year_of_capacity_data',
                      'generation_data_source',
                      'generation_gwh_2018',
                      'generation_gwh_2019',
                      'estimated_generation_note_2013',
                      'estimated_generation_note_2014',
                      'estimated_generation_note_2015',
                      'estimated_generation_note_2016',
                      'estimated_generation_note_2017']
  ppg.drop(unwanted_columns, axis=1, inplace=True)

  unwanted_columns = ['No.', 'Number of Units', 'Total Plant Installed Capacity (MW)']
  ppe.drop(unwanted_columns, axis=1, inplace=True)

  # AGGREGATING DATA
  avgs = ['generation_gwh_2013',
          'generation_gwh_2014',
          'generation_gwh_2015',
          'generation_gwh_2016',
          'generation_gwh_2017']
  ppg['AVG_GENERATION'] = ppg[avgs].mean(axis=1)

  avgs = ['estimated_generation_gwh_2013',
          'estimated_generation_gwh_2014',
          'estimated_generation_gwh_2015',
          'estimated_generation_gwh_2016',
          'estimated_generation_gwh_2017']
  ppg['AVG_EST_GENERATION'] = ppg[avgs].mean(axis=1)

  # merge the two average columns into a single column
  ppg['GENERATION_MW'] = ppg.apply(lambda x : np.fmax(x['AVG_GENERATION'], x['AVG_EST_GENERATION']), axis=1)

  # remove unaggregated columns
  unwanted_columns = ['AVG_GENERATION',
                      'AVG_EST_GENERATION',
                      'generation_gwh_2013',
                      'generation_gwh_2014',
                      'generation_gwh_2015',
                      'generation_gwh_2016',
                      'generation_gwh_2017',
                      'estimated_generation_gwh_2013',
                      'estimated_generation_gwh_2014',
                      'estimated_generation_gwh_2015',
                      'estimated_generation_gwh_2016',
                      'estimated_generation_gwh_2017']
  ppg.drop(unwanted_columns, axis=1, inplace=True)

  # merge rows by fuel type
  generation_dist = ppg.groupby('primary_fuel')['GENERATION_MW'].sum().sort_values(ascending=False)
  emission_dist = ppe.groupby('Fuel Types').aggregate({'CO2 Emissions (Mg)':'sum',
                                                       'SO2 Emissions (Mg)':'sum',
                                                       'NOx Emissions (Mg)':'sum',
                                                       'PM2.5 Emissions (Mg)':'sum'})

  # COMBINING TABLES
  generation_dist.index = generation_dist.index.str.upper()
  generation_dist.drop('WAVE AND TIDAL', axis=0, inplace=True)
  gen_other = ['PETCOKE','WASTE','COGENERATION','STORAGE','NUCLEAR']
  generation_dist['OTHER'] = generation_dist[gen_other].sum(axis=0)
  generation_dist.drop(gen_other, axis=0, inplace=True)
  emission_dist = emission_dist.rename(index={'NG':'GAS'})

  power = pd.concat([generation_dist, emission_dist], axis=1)
  power = power.fillna(0)

  return power

############ Function Call ############
global_power()

,GENERATION_MW,CO2 Emissions (Mg),SO2 Emissions (Mg),NOx Emissions (Mg),PM2.5 Emissions (Mg)
COAL,9.960694e+06,8.880799e+09,2.973419e+07,1.848038e+07,2.508320e+06
GAS,6.304916e+06,2.518846e+09,5.228739e+04,3.440040e+06,4.144760e+04
HYDRO,3.755360e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
WIND,7.094421e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
OIL,5.360917e+05,7.370139e+08,8.689335e+06,2.723445e+06,9.304416e+04
SOLAR,3.486002e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GEOTHERMAL,6.083382e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
BIOMASS,3.368912e+04,1.196067e+08,1.792376e+05,1.744930e+05,3.713316e+04
OTHER,3.011469e+06,2.749785e+08,1.500827e+05,3.382073e+05,1.042250e+04


## Web Collection Requirement \#1


In [4]:
def web_scrape(): #overarching function - data frame doesnt present as nicely as when on its own which is why the code is broken up 
    #below. Not sure if we need just one function because that works just as well up until you call function which returns None

    hashlist = []
    hashdict = {}
    url = requests.get('https://whattomine.com/gpus')
    soup = BeautifulSoup(url.text, 'html.parser')
    for hasher in soup.find_all('div',{'class' :'position-relative'}):
        for h in hasher.stripped_strings:
            hashlist.append(h)
            hashlist2=  hashlist[::2]
            hashdict['Hashrate(Millions of Hash Per Sec)'] = [z for z in hashlist2] #creates a dict that connects to a list of all the HashRates of GPUs for easy iteration when creating visuals
    print(hashdict)

    revlist = []
    revdict = {}
    for rev in soup.find_all('td',{'class':'text-right table-success font-weight-bold'}):
        for r in rev.stripped_strings:
            revlist.append(r)
            revdict["24Hour Revenue"] = [z for z in revlist]
    print(revdict) #creates a dict that connects to a list of all the HashRates of GPUs for easy iteration when creating visuals
    
    namelist = []
    namedict = {}
    for name in soup.find_all('td'):
        for n in name.stripped_strings:
            namelist.append(n)
            namelist2 = namelist[1:64:16]
            namelist2+=namelist[66:99:16]
            namelist2+=namelist[115:175:16]
            namelist2+=namelist[180:181]
            namelist2+=namelist[196:260:16]
            namelist2+=namelist[261:262]
            namelist2+=namelist[277:325:16]
            namelist2+=namelist[326:343:16]
            namelist2+=namelist[359:377:17]
            namelist2+=namelist[392:521:16]
            namelist2+=namelist[537:650:16]
            #no identifiable iteration order, easier for me to do it mathematically this way
            namedict["GPU Model"] = [z for z in namelist2]
    print(namedict)
    
    fulldict = {}
    for i in range(len(namelist2)):
        fulldict[namelist2[i]] = {"Hashrate(Millions of Hash Per Sec)":hashlist2[i],"24Hour Revenue": revlist[i]}
    print(fulldict) # this dictionary matches GPU model to the hash rate and 24 hour revenue
    
    data = []
    data.append(hashlist2)
    data.append(revlist)
    df = pd.DataFrame(data, index= ['Hashrate (Millions of Hashes Per Sec)','24 Hour Revenue'], columns = namelist2).T
    return df #creates a data frame with the columns as GPU name and the index as the description 

############ Function Call ############
web_scrape()

## Web Collection Requirement \#2

In [5]:
def web_parser2():
  pass





############ Function Call ############
web_parser2()

## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [6]:
def extra_source1():
    pass

    
############ Function Call ############
extra_source1()

In [7]:
# Define further extra source functions as necessary

# Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. 

2. 

3. 

4. (if applicable)

5. (if applicable)
